In [2]:
import pandas as pd
import numpy as np
# from keras.models import load_model
import h5py
import pandas as pd
import argparse
import SimpleITK as sitk
from PIL import Image
import os, glob 
import os, os.path
import tensorflow as tf
import keras
from UNET_utils import *
%matplotlib inline

In [3]:
# import argparse
# parser = argparse.ArgumentParser(description='Prediction on HOLDOUT subset',add_help=True)
# parser.add_argument("--holdout", type=int, default=0, help="HOLDOUT subset for predictions")
# args = parser.parse_args()
# HOLDOUT = args.holdout

In [4]:
HOLDOUT = 5
HO_dir = 'HO{}/'.format(HOLDOUT)
data_dir = '../data/luna16/'
model_wghts = 'hdf5/UNET_weights_H{}.h5'.format(HOLDOUT)

# img_filename = '1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437.mhd'
# model_file = 'hdf5/cnn_3DUNET_64_64_64_HOLDOUT1_20180420_120442.hdf5'
# c_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef}
# model = load_model(data_dir + model_file, custom_objects=c_objects)

In [5]:
def size_equal(s1, s2):
    return sorted(s1) == sorted(s2)

In [6]:
def model_create_load(padded_img):
    input_shape = padded_img.reshape(tuple(list (padded_img.shape) + [1])).shape
    model = create_UNET3D(input_shape)

    model.compile(optimizer='adam',
                  loss={'PredictionMask': dice_coef_loss, \
                        'PredictionClass': 'binary_crossentropy'}, \
                  loss_weights={'PredictionMask': 0.8, 'PredictionClass': 0.2},
                  metrics={'PredictionMask':dice_coef,'PredictionClass': 'accuracy'})

    # print(tmp_model.summary())
    model.load_weights(data_dir + model_wghts, by_name=True)
    return model

In [7]:
padded_size = (480, 480, 368)
predictions_dict = {}
size_dict = {}
prev_img_size = (0,0,0)

for f in glob.glob(data_dir + HO_dir + '*.mhd'):
    print ("\n Processing scan file: {}".format(f))
    
    itk_img = sitk.ReadImage(f) 
    img_np_array = sitk.GetArrayFromImage(itk_img)
    original_size = img_np_array.shape
    print ("Original-Size of loaded image : {}".format(original_size))
    
    ## Normalizing the image size ...Need to confirm?  AL
    itk_img_norm = normalize_img(itk_img)
    img_np_array_norm = sitk.GetArrayFromImage(itk_img_norm)
    normalized_size = img_np_array_norm.shape
#     print ("Normalized-Size of image : {}".format(normalized_size))
    
    # Normalizing HU of image
    img = normalize_HU(img_np_array_norm)
    img = np.swapaxes(img, 0,2)
    print ("Swapped axis input image : {}".format(img.shape))
    
    padded_img = np.zeros(padded_size)
    padded_img[ :img.shape[0], :img.shape[1], :img.shape[2] ] = img
    
    model = model_create_load(padded_img)      
    padded_img = padded_img.reshape(tuple([1] + list (padded_img.shape) + [1]))
    print ("Size of input padded image : {}".format(padded_img.shape))

    #### Owing to large image size requiring heaving processing my m/c hangs
    #### For predictions - uncomment the following line on AWS or GPU m/c
#     f_predictions = model.predict(padded_img, verbose=1)
#     predictions_dict[f] = f_predictions
    size_dict[f] = img.shape


 Processing scan file: ../data/luna16/HO5/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437.mhd
Original-Size of loaded image : (117, 512, 512)
Normalized-Size of image : (292, 400, 400)
Swapped axis input image : (400, 400, 292)
Size of input padded image : (1, 480, 480, 368, 1)

 Processing scan file: ../data/luna16/HO5/1.3.6.1.4.1.14519.5.2.1.6279.6001.278660284797073139172446973682.mhd
Original-Size of loaded image : (117, 512, 512)
Normalized-Size of image : (292, 340, 340)
Swapped axis input image : (340, 340, 292)
Size of input padded image : (1, 480, 480, 368, 1)


In [8]:
size_dict

{'../data/luna16/HO5/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437.mhd': (400,
  400,
  292),
 '../data/luna16/HO5/1.3.6.1.4.1.14519.5.2.1.6279.6001.278660284797073139172446973682.mhd': (340,
  340,
  292)}

#### Testing with image of size 48,48,48 initialized with random between 0 and 1

In [9]:
small_img = np.random.rand(48,48,48)
small_img = small_img.reshape(tuple([1] + list (small_img.shape) + [1]))
# small_img.shape 
small_shape = (48, 48, 48, 1)
model = create_UNET3D(small_shape)
model.compile(optimizer='adam',
              loss={'PredictionMask': dice_coef_loss, \
                    'PredictionClass': 'binary_crossentropy'}, \
              loss_weights={'PredictionMask': 0.8, 'PredictionClass': 0.2},
              metrics={'PredictionMask':dice_coef,'PredictionClass': 'accuracy'})
# print(tmp_model.summary())
model.load_weights(data_dir + model_wghts, by_name=True)

In [10]:
predictions_small_img = model.predict(small_img, verbose=1)
result = pd.DataFrame(predictions_small_img)

1/1 [==============================] - 9s 9s/step


In [ ]:
# print ("Shape of predicted mask or segmented image : {}".format(predictions_small_img[0].shape))
# print ("Shape of predicted class : {}".format(predictions_small_img[1].shape))
# predictions_small_img[0] [:, 25 : 26, :]

In [ ]:
# ## AL - TEST : making an image of size 48,48,48 with random 0 or 1
# ### Case 2 : As a test created an input image of size (1, 48,48,48,1) 
# # with random 0 or 1; this works fine and able to create predictions successfully
# t2 =  np.random.choice(2,(48,48,48))
# t2 = t2.reshape(tuple([1] + list (t2.shape) + [1]))

# print ("Shape of test input image : {}".format(t2.shape))
# predictions = model.predict(t2, verbose=2)

# print ("Shape of predicted mask or segmented image : {}".format(predictions[0].shape))
# print ("Shape of predicted class : {}".format(predictions[1].shape))
# # predictions[0] [:, 25 : 26, :]